In [7]:
import logging
import pickle
from pathlib import Path
from uuid import uuid4

import chromadb
import numpy as np


In [8]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Creation

In [9]:
# directory, where Chroma will store collections
CHROMADB_STORAGE_FOLDER = Path("/home/deniskirbaba/Documents/legal-ai/storage")

# directory with data, used to init legal practice collection (i.e. documents, embeddings, metadata)
LEGAL_DOCS_DATA_FOLDER = Path("./data")

In [10]:
def init_legal_docs_db():
    """
    Initializes ChromaDB vector storage with legal docs.
    """
    persistent_client = chromadb.PersistentClient(path=str(CHROMADB_STORAGE_FOLDER))
    collection = persistent_client.create_collection(
        name="legal_docs", get_or_create=True, metadata={"hnsw:space": "cosine"}
    )

    # Load embeddings
    logging.info("Start loading embeddings.")
    embs = np.load(LEGAL_DOCS_DATA_FOLDER / "chunks_emb.npy")
    logging.info("Successfully load embeddings.")

    # Load chunks text and metadata (dict with docs titles)
    logging.info("Start loading docs and metadatas.")
    with open(LEGAL_DOCS_DATA_FOLDER / "chunks.pkl", "rb") as f:
        chunks = pickle.load(f)
    logging.info("Successfully load docs and metadatas.")

    # Add data to the collection
    logging.info("Creating collection.")
    collection.add(
        ids=[str(uuid4()) for _ in range(len(embs))],
        embeddings=embs,
        metadatas=[ch.metadata for ch in chunks],
        documents=[ch.page_content for ch in chunks],
    )

    return persistent_client, collection

In [17]:
%%time
chroma_client, legal_docs_collection = init_legal_docs_db()

INFO:root:Start loading embeddings.


INFO:root:Successfully load embeddings.
INFO:root:Start loading docs and metadatas.
INFO:root:Successfully load docs and metadatas.
INFO:root:Creating collection.


CPU times: user 1min 8s, sys: 8.28 s, total: 1min 17s
Wall time: 1min 23s


## Testing

In [20]:
chroma_client.list_collections()

[Collection(name=legal_docs)]

In [21]:
legal_docs_collection.count()

17285

In [23]:
legal_docs_collection.get('baedf80e-0966-41a0-90c2-663bc1acec52')

{'ids': ['baedf80e-0966-41a0-90c2-663bc1acec52'],
 'embeddings': None,
 'documents': ['РАЗДЕЛ ПЕРВЫЙ\n\n\nГЛАВА 1.\n\nОСНОВЫ КОНСТИТУЦИОННОГО СТРОЯ\n\n\nСтатья 1\n\n\n1. Российская Федерация - Россия есть демократическое федеративное правовое государство с республиканской формой правления.\n\n2. Наименования Российская Федерация и Россия равнозначны.\n\n\nСтатья 2\n\n\nЧеловек, его права и свободы являются высшей ценностью. Признание, соблюдение и защита прав и свобод человека и гражданина - обязанность государства.\n\n\nСтатья 3\n\n\n1. Носителем суверенитета и единственным источником власти в Российской Федерации является ее многонациональный народ.\n\n2. Народ осуществляет свою власть непосредственно, а также через органы государственной власти и органы местного самоуправления.\n\n3. Высшим непосредственным выражением власти народа являются референдум и свободные выборы.\n\n4. Никто не может присваивать власть в Российской Федерации. Захват власти или присвоение властных полномочий 